##### Copyright 2019 The TensorFlow Authors.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初学者的 TensorFlow 2.0 教程

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.google.cn/tutorials/quickstart/beginner"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png" />在 TensorFlow.org 观看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/quickstart/beginner.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png" />在 Google Colab 运行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/quickstart/beginner.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" />在 GitHub 查看源代码</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/quickstart/beginner.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png" />下载笔记本</a>
  </td>
</table>

Note: 我们的 TensorFlow 社区翻译了这些文档。因为社区翻译是尽力而为， 所以无法保证它们是最准确的，并且反映了最新的
[官方英文文档](https://tensorflow.google.cn/?hl=en)。如果您有改进此翻译的建议， 请提交 pull request 到
[tensorflow/docs](https://github.com/tensorflow/docs) GitHub 仓库。要志愿地撰写或者审核译文，请加入
[docs-zh-cn@tensorflow.org Google Group](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-zh-cn)。

这是一个 [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) 笔记本文件。 Python程序可以直接在浏览器中运行，这是学习 Tensorflow 的绝佳方式。想要学习该教程，请点击此页面顶部的按钮，在Google Colab中运行笔记本。

1. 在 Colab中, 连接到Python运行环境： 在菜单条的右上方, 选择 *CONNECT*。
2. 运行所有的代码块: 选择 *Runtime* > *Run all*。

下载并安装 TensorFlow 2.0 测试版包。将 TensorFlow 载入你的程序：

In [1]:
# 安装 TensorFlow

import tensorflow as tf

载入并准备好 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/)。将样本从整数转换为浮点数：

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

将模型的各层堆叠起来，以搭建 `tf.keras.Sequential` 模型。为训练选择优化器和损失函数：

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

训练并验证模型：

In [4]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 4s 1ms/step - loss: 0.2872 - accuracy: 0.9158
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1401 - accuracy: 0.9599
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1061 - accuracy: 0.9681
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0854 - accuracy: 0.9736
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0750 - accuracy: 0.9763
313/313 - 0s - loss: 0.0731 - accuracy: 0.9788 - 411ms/epoch - 1ms/step


[0.07310659438371658, 0.9787999987602234]

现在，这个照片分类器的准确度已经达到 98%。想要了解更多，请阅读 [TensorFlow 教程](https://tensorflow.google.cn/tutorials/)。

### 保存模型

In [13]:
h5_model_name = 'Dense-Mnist-int8.h5'
model.save(h5_model_name)

### 将模型量化为TFLite(Int8)

In [15]:
tflite_model_name = 'Dense-Mnist-int8.tflite'
def representative_data_gen():#量化校准数据集
    train_images = x_test
    for image in train_images[0:300,:,:]:
        yield[image.reshape(-1,train_images.shape[1],train_images.shape[2]).astype("float32")]
        
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(h5_model_name)
#输入/输出 Tensor类型
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

tflite_model = converter.convert()

with open(tflite_model_name, "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\xddcore\AppData\Local\Temp\tmpl9ebjoci\assets


INFO:tensorflow:Assets written to: C:\Users\xddcore\AppData\Local\Temp\tmpl9ebjoci\assets


INFO:tensorflow:Restoring parameters from C:\Users\xddcore\AppData\Local\Temp\tmpl9ebjoci\variables\variables


INFO:tensorflow:Restoring parameters from C:\Users\xddcore\AppData\Local\Temp\tmpl9ebjoci\variables\variables


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}


...

INFO:tensorflow:Tensor's key in saved_model's tensor_map: dense_1


INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 10), type: DT_FLOAT


INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 10), type: DT_FLOAT


INFO:tensorflow:Restoring parameters from C:\Users\xddcore\AppData\Local\Temp\tmpl9ebjoci\variables\variables


INFO:tensorflow:Restoring parameters from C:\Users\xddcore\AppData\Local\Temp\tmpl9ebjoci\variables\variables


### 评估量化后的tflite

In [16]:
def evaluate(interpreter_path):
    test_images = (x_test*255)-128
    test_labels = y_test
    #加载模型并分配张量
    interpreter = tf.lite.Interpreter(model_path=interpreter_path)
    interpreter.allocate_tensors()

    #获得输入输出张量.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    import numpy as np
    index = input_details[0]['index']
    shape = input_details[0]['shape']
    acc_count = 0
    image_count = test_images.shape[0]
    for i in range(image_count):
        interpreter.set_tensor(index, test_images[i].reshape(shape).astype("int8"))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        #print(output_data)
        label = np.argmax(output_data)
        if label == test_labels[i]:
            acc_count += 1
    print("Test Dataset Accuracy is {:.2%}".format(acc_count/(image_count)))
evaluate(tflite_model_name)

Test Dataset Accuracy is 97.85%


#### 导出模型的相关信息

In [7]:
print(model.get_config())

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 28, 28), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'flatten_input'}}, {'class_name': 'Flatten', 'config': {'name': 'flatten', 'trainable': True, 'batch_input_shape': (None, 28, 28), 'dtype': 'float32', 'data_format': 'channels_last'}}, {'class_name': 'Dense', 'config': {'name': 'dense', 'trainable': True, 'dtype': 'float32', 'units': 128, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}}, {'class_name': 'Dropout', 'config': {'name': 'dropout', 'trainable': True, 'dtype': 'float32', 'rate': 0.2, 'noise_shape': None, 'seed': None}}, {'class_name': 'Dense', 'config': {'name': 'dense_1', 'trainable': True, '

In [15]:
weights = model.get_weights()

print(weights[0].shape)#第一层dense的weights
print(weights[1].shape)#第一层dense的bias

print(weights[2].shape)#第二层dense的weights
print(weights[3].shape)#第二层dense的bias


(784, 128)
(128,)
(128, 10)
(10,)


### 导出每一层的权重和偏置(以下代码废弃，权重提取操作使用OpenNNA_Model_Converter完成。

In [26]:
import numpy as np

#提取第一层Dense的weights
dense1_txt = "./dense_1_weights.txt"
dense1_weights = np.swapaxes(weights[0],0,1)
txtfile_1 = open(dense1_txt, 'w', encoding='UTF-8')
txtfile_1.write("\n{")
for i in range(0,128):
    txtfile_1.write("\n\t{")
    for j in range(0,784):
        txtfile_1.write("%8.6f"%dense1_weights[i][j]+",")
    txtfile_1.write("},\n")
txtfile_1.write("\n}")
txtfile_1.close()

#提取第一层Dense的bias
dense2_txt = "./dense_1_bias.txt"
txtfile_1 = open(dense2_txt, 'w', encoding='UTF-8')
txtfile_1.write("\n{")
for i in range(0,128):
    txtfile_1.write("%8.6f"%weights[1][i]+",")
txtfile_1.write("}")
txtfile_1.close()

#提取第二层Dense的weights
dense3_txt = "./dense_2_weights.txt"
dense2_weights = np.swapaxes(weights[2],0,1)
txtfile_1 = open(dense3_txt, 'w', encoding='UTF-8')
txtfile_1.write("\n{")
for i in range(0,10):
    txtfile_1.write("\n\t{")
    for j in range(0,128):
        txtfile_1.write("%8.6f"%dense2_weights[i][j]+",")
    txtfile_1.write("},\n")
txtfile_1.write("\n}")
txtfile_1.close()

#提取第二层Dense的bias
dense4_txt = "./dense_2_bias.txt"
txtfile_1 = open(dense4_txt, 'w', encoding='UTF-8')
txtfile_1.write("\n{")
for i in range(0,10):
    txtfile_1.write("%8.6f"%weights[3][i]+",")
txtfile_1.write("}")
txtfile_1.close()


### 下载Mnist数据集，并转换为.jpg文件存储在文件夹下

In [30]:
import cv2

import os

from keras.datasets import mnist

 

import numpy as np

str_1 = 'mnist_train'

str_2 = 'mnist_test'

if os.path.exists(str_1) is False:

    os.mkdir(str_1)

 

if os.path.exists(str_2) is False:

    os.mkdir(str_2)

#自动下载mnist数据集

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

 

for i in range(0, 59999):  # 迭代 0 到 59999 之间的数字

    fileName = str_1+"/"+ str(Y_train[i]) + "_" + str(i) + ".jpg"

    cv2.imwrite(fileName, X_train[i])

 

for i in range(0, 9999):  # 迭代 0 到 9999 之间的数字

    fileName = str_2+"/"+ str(Y_test[i]) + "_" + str(i) + ".jpg"

    cv2.imwrite(fileName, X_test[i])

### 将MNIST的.jpg转换为c 数组存到.txt中

In [28]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models, layers, optimizers
import numpy as np

image_value = tf.io.read_file('./mnist_train/5_0.jpg')

#解码为tensor
image_value = tf.io.decode_jpeg(image_value,channels = 1)


#image_value = tf.image.resize(image_value, (32,32))#改变像素值为32*32


#tensor转array
image_value = image_value.numpy()
loadData1 = np.swapaxes(image_value, 0, 2)
loadData2 = np.swapaxes(loadData1, 1, 2)

row = 28
col = 28

txtfile = open(r'image_5_0_int8.txt', 'w',encoding='UTF-8')
for i in range(0,1):
    txtfile.write("\t{\n")
    for j in range(0,row):
        txtfile.write("\t\t{")
        for k in range(0,col):
            #txtfile.write(str(loadData2[i][j][k])+", ") 
            txtfile.write("%d"%(loadData2[i][j][k])+", ") 
        txtfile.write("},\n")
    txtfile.write("\t},\n\n")
txtfile.close()